In [ ]:
# Cell 1: Setup and Imports

# Core Libraries for data handling and numerical operations
import pandas as pd
import numpy as np
import os 
from datetime import datetime # For handling timestamp data

# Scikit-learn for data scaling (will be used later)
from sklearn.preprocessing import MinMaxScaler

# Optional: Ignore warnings for cleaner output during execution
import warnings
warnings.filterwarnings('ignore')

# Print statement to confirm execution
print("Libraries imported successfully.")

Libraries imported successfully.


In [ ]:
# Cell 2: Load JSON, Process Keys, and Create Flat DataFrame 

import pandas as pd
import numpy as np
import os
import json 
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# --- Configuration ---
DATA_DIR = '/workspace/Data' # Assuming files are in the root project folder
FILE_1 = 'compoundV2_transactions_ethereum_chunk_0.json'
FILE_2 = 'compoundV2_transactions_ethereum_chunk_1.json'
FILE_3 = 'compoundV2_transactions_ethereum_chunk_2.json' # Include the problematic file

json_files = [os.path.join(DATA_DIR, f) for f in [FILE_1, FILE_2, FILE_3]]
print(f"Attempting to load and process JSON files: {json_files}")

# --- Define Mapping from JSON Keys to Standard TX Types ---
 # Keys confirmed present in the loaded data segments
KEY_TO_TX_TYPE_MAP = {
    'deposits': 'Deposit',
    'withdraws': 'Withdraw', # Assuming 'withdraws' is the key for withdrawals
    'borrows': 'Borrow',     # Assuming 'borrows' is the key
    'repays': 'Repay',       # Assuming 'repays' is the key
    # 'liquidations' key was checked for but not found in these files.
}

all_transactions = [] # Master list to hold ALL flattened transactions from ALL files

# --- Load and Process Data File by File ---
loaded_files_count = 0
for f in json_files:
    print(f"Processing {f}...")
    try:
        with open(f, 'r', encoding='utf-8') as file: 
            data = json.load(file) # Load the entire JSON structure into a Python dict

        if not isinstance(data, dict):
            print(f"  Warning: Expected a dictionary structure in {f}, but got {type(data)}. Skipping file.")
            continue

        file_tx_count = 0
        # Iterate through the keys we care about (deposits, withdraws, etc.)
        for key, tx_type in KEY_TO_TX_TYPE_MAP.items():
            if key in data and isinstance(data[key], list):
                transaction_list = data[key]
                # print(f"  Processing {len(transaction_list)} transactions from key '{key}'...") # Debug print for checking..

                # --- Flattening logic embedded here ---
                for record in transaction_list:
                    try:
                        # Extract fields 
                        wallet = record.get('account', {}).get('id')
                        timestamp = record.get('timestamp')
                        amount = record.get('amount')
                        # Handle potentially nested amount field
                        if isinstance(amount, dict):
                           amount = amount.get('amount') # Example, adjust key if needed

                        asset = record.get('asset', {}).get('symbol')
                        tx_hash = record.get('id') # Transaction specific id/hash
                        amount_usd = record.get('amountUSD')

                        if wallet and timestamp is not None and amount is not None:
                             all_transactions.append({
                                 'wallet_address': wallet,
                                 'timestamp': timestamp,
                                 'tx_hash': tx_hash,
                                 'std_tx_type': tx_type, # Use the mapped type
                                 'amount': amount,
                                 'amountUSD': amount_usd,
                                 'asset_symbol': asset
                             })
                             file_tx_count += 1
                        # else: # Optional not implemented: Debugging for missing essential data
                        #    print(f"    Skipping record due to missing essential data: Wallet={wallet}, TS={timestamp}, Amt={amount}")

                    except Exception as e_rec:
                        # Log error processing a specific record but continue with others
                        # print(f"    Warning: Error processing record within {key}: {e_rec}. Record: {record}")
                        pass
            # else: # Optional not implemented : Debugging for missing keys
                # print(f"  Key '{key}' not found or not a list in {f}.")


        print(f"  Successfully processed {file_tx_count} transactions from {f}.")
        loaded_files_count += 1

    except FileNotFoundError:
        print(f"---!!! ERROR: File not found - {f}. Please check the filename and DATA_DIR setting. !!!---")
    except json.JSONDecodeError as e_json:
        print(f"---!!! ERROR: Invalid JSON structure in {f}: {e_json} !!!---")
    except Exception as e:
        print(f"---!!! ERROR processing {f}: {e} !!!---")
        # Optional implementation to raise e to stop execution on error

# --- Create Final DataFrame ---
if all_transactions:
    print(f"\nCreating final DataFrame from {len(all_transactions):,} processed transactions...")
    df_final = pd.DataFrame(all_transactions)
    print("DataFrame created successfully.")

    # --- Initial Inspection of the FLAT DataFrame ---
    print("\n--- First 5 rows of combined data (df_final.head()): ---")
    print(df_final.head())

    print("\n--- DataFrame Info (df_final.info()): ---")
    pd.options.display.max_columns = None
    df_final.info(verbose=True, show_counts=True)

    # Check memory usage
    print("\n--- Memory Usage ---")
    print(df_final.memory_usage(deep=True).sum() / (1024**2), "MB")

else:
    print("\n---!!! No transactions were processed. Check file paths, JSON structure, and previous errors. Cannot proceed. !!!---")
    raise ValueError("No data processed. Halting execution.")

Attempting to load and process JSON files: ['/workspace/Data/compoundV2_transactions_ethereum_chunk_0.json', '/workspace/Data/compoundV2_transactions_ethereum_chunk_1.json', '/workspace/Data/compoundV2_transactions_ethereum_chunk_2.json']
Processing /workspace/Data/compoundV2_transactions_ethereum_chunk_0.json...
  Successfully processed 40000 transactions from /workspace/Data/compoundV2_transactions_ethereum_chunk_0.json.
Processing /workspace/Data/compoundV2_transactions_ethereum_chunk_1.json...
  Successfully processed 40000 transactions from /workspace/Data/compoundV2_transactions_ethereum_chunk_1.json.
Processing /workspace/Data/compoundV2_transactions_ethereum_chunk_2.json...
  Successfully processed 40000 transactions from /workspace/Data/compoundV2_transactions_ethereum_chunk_2.json.

Creating final DataFrame from 120,000 processed transactions...
DataFrame created successfully.

--- First 5 rows of combined data (df_final.head()): ---
                               wallet_addr

In [ ]:
# Cell 3: Data Cleaning & Type Conversion


# Let's rename df_final to df for simplicity in subsequent cells
df = df_final.copy() 
del df_final #  clean up memory if df_final is large

print("Working with DataFrame 'df'.")
print(f"Initial rows: {len(df):,}")

# --- 1. Convert Timestamp ---
print("\nConverting 'timestamp' column...")
# The values look like Unix timestamps (seconds since epoch). Convert to datetime.
df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce') # Convert to numeric first
df.dropna(subset=['timestamp'], inplace=True) # Drop rows where conversion failed
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s') # Convert numeric seconds to datetime
print("Timestamp conversion complete.")

# --- 2. Convert Numerical Amounts ---
print("Converting 'amount' and 'amountUSD' columns...")
# Convert amount and amountUSD to numeric types.
# 'coerce' will turn unconvertible values into NaN (Not a Number).
df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
df['amountUSD'] = pd.to_numeric(df['amountUSD'], errors='coerce')

# --- 3. Handle Missing Numerical Values ---
# Check how many NaNs were introduced during conversion
print(f"NaNs in 'amount' after conversion: {df['amount'].isnull().sum()}")
print(f"NaNs in 'amountUSD' after conversion: {df['amountUSD'].isnull().sum()}")

# Strategy: For scoring, amountUSD is more important. If amountUSD is NaN,
# the transaction's value impact is unknown. We could:
#   a) Fill with 0: Assumes no value, might understate risk/contribution.
#   b) Drop row: Removes the transaction entirely. Safer if value is critical.
#   c) Impute (median/mean): Complex, might not be suitable for tx data.
# Let's choose 'drop row' for amountUSD as it's crucial for value-based features.
# We can be more lenient with 'amount' if needed, maybe fill with 0, but for Version 1 just drop the row if amountUSD is NaN

initial_rows = len(df)
df.dropna(subset=['amountUSD'], inplace=True)
print(f"Dropped {initial_rows - len(df):,} rows due to missing 'amountUSD'.")
# Optional: if we need to handle NaNs in 'amount' differently, e.g., fill with 0
# df['amount'].fillna(0, inplace=True)

print("Numerical amount conversion and NaN handling complete.")


# --- 4. Standardize Wallet Address ---
print("Standardizing 'wallet_address' column...")
# Ensureing consistent formatting (e.g., lowercase)
df['wallet_address'] = df['wallet_address'].astype(str).str.lower()
print("Wallet address standardization complete.")


# --- 5. Verify Transaction Types ---
print("\nVerifying 'std_tx_type' values...")
print("Unique transaction types found:")
print(df['std_tx_type'].unique())
# Check value counts
print("\nTransaction type distribution:")
print(df['std_tx_type'].value_counts())


# --- Final Check ---
print(f"\nDataFrame shape after cleaning: {df.shape}")
print("\n--- DataFrame Info after Cleaning (df.info()): ---")
df.info(verbose=True, show_counts=True)

print("\n--- First 5 rows after Cleaning (df.head()): ---")
print(df.head())

Working with DataFrame 'df'.
Initial rows: 120,000

Converting 'timestamp' column...
Timestamp conversion complete.
Converting 'amount' and 'amountUSD' columns...
NaNs in 'amount' after conversion: 0
NaNs in 'amountUSD' after conversion: 0
Dropped 0 rows due to missing 'amountUSD'.
Numerical amount conversion and NaN handling complete.
Standardizing 'wallet_address' column...
Wallet address standardization complete.

Verifying 'std_tx_type' values...
Unique transaction types found:
['Deposit' 'Withdraw' 'Borrow' 'Repay']

Transaction type distribution:
std_tx_type
Deposit     30000
Withdraw    30000
Borrow      30000
Repay       30000
Name: count, dtype: int64

DataFrame shape after cleaning: (120000, 7)

--- DataFrame Info after Cleaning (df.info()): ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   wallet_addres

In [ ]:
# Cell 4: Feature Engineering

print("Starting Feature Engineering...")
print(f"Input DataFrame 'df' shape: {df.shape}")

# Sort by wallet and time - useful for some potential future window functions
df.sort_values(by=['wallet_address', 'timestamp'], inplace=True)

# --- Define Aggregation Functions ---
# We will group by wallet_address and calculate various stats
agg_funcs = {
    'timestamp': ['min', 'max', 'nunique'], # first seen, last seen, count unique timestamps (proxy for active days)
    'tx_hash': ['count'],           # Total number of transactions for the wallet
    'asset_symbol': ['nunique']     # Number of unique assets interacted with
    # We will calculate type-specific counts and sums separately for clarity
}

# --- Perform Initial Grouping ---
print("Grouping data by wallet address...")
wallet_features = df.groupby('wallet_address').agg(agg_funcs)

# Flatten multi-index columns created by .agg()
wallet_features.columns = ['_'.join(col).strip('_') for col in wallet_features.columns.values]

# Rename columns for better readability
wallet_features.rename(columns={
    'timestamp_min': 'first_seen',
    'timestamp_max': 'last_seen',
    'timestamp_nunique': 'unique_timestamps_count', # Can refine to days later if needed
    'tx_hash_count': 'total_transactions',
    'asset_symbol_nunique': 'unique_assets_interacted'
}, inplace=True)

print("Initial aggregation complete.")

# --- Calculate Time-Based Features ---
print("Calculating time-based features...")
# Calculate wallet lifespan in days. Add 1 to avoid zero lifespan for single-day activity.
wallet_features['wallet_lifespan_days'] = (wallet_features['last_seen'] - wallet_features['first_seen']).dt.days + 1

# Calculate transactions per day
wallet_features['transactions_per_day'] = wallet_features['total_transactions'] / wallet_features['wallet_lifespan_days']
print("Time-based features calculated.")


# --- Calculate Counts and USD Amounts per Transaction Type ---
print("Calculating counts and sums per transaction type...")
# Use pivot_table for efficient aggregation of counts and sums based on 'std_tx_type'

# Counts per type
action_counts = pd.pivot_table(df, index='wallet_address', columns='std_tx_type', aggfunc={'tx_hash': 'count'}, fill_value=0)
action_counts.columns = [f'{col[1]}_count' for col in action_counts.columns] # Flatten column names

# Sums of amountUSD per type
action_sums_usd = pd.pivot_table(df, index='wallet_address', columns='std_tx_type', values='amountUSD', aggfunc='sum', fill_value=0)
action_sums_usd.columns = [f'{col}_amountUSD_sum' for col in action_sums_usd.columns] # Flatten column names

# Average amountUSD per type (handle potential division by zero if counts are zero)
# We can calculate this later after merging counts and sums

# --- Merge Action Counts and Sums back into wallet_features ---
print("Merging aggregated features...")
wallet_features = wallet_features.join(action_counts, how='left')
wallet_features = wallet_features.join(action_sums_usd, how='left')

# Fill NaNs that might arise from the join (though 'left' join on index should align) with 0
wallet_features.fillna(0, inplace=True) # Fill potential NaNs in count/sum columns with 0
print("Aggregated features merged.")


# --- Calculate Derived Behavioral Features (using amountUSD) ---
print("Calculating derived behavioral features...")

# Ensure required columns exist, using .get() with default 0 for safety
deposit_sum_col = 'Deposit_amountUSD_sum'
withdraw_sum_col = 'Withdraw_amountUSD_sum'
borrow_sum_col = 'Borrow_amountUSD_sum'
repay_sum_col = 'Repay_amountUSD_sum'
borrow_count_col = 'Borrow_count'

# Net Deposit Amount (USD)
wallet_features['net_deposit_amountUSD'] = wallet_features.get(deposit_sum_col, 0) - wallet_features.get(withdraw_sum_col, 0)

# Repayment Ratio (USD)
# How much of the total borrowed USD value has been repaid?
epsilon = 1e-6 # Small value to prevent division by zero
total_borrowed_usd = wallet_features.get(borrow_sum_col, 0)
total_repaid_usd = wallet_features.get(repay_sum_col, 0)
wallet_features['repayment_ratio_usd'] = total_repaid_usd / (total_borrowed_usd + epsilon)
# Cap ratio at 1 (can't repay more than borrowed in this simple model)
wallet_features['repayment_ratio_usd'] = wallet_features['repayment_ratio_usd'].clip(0, 1)

# Borrow-to-Deposit Ratio (USD) - Proxy for Leverage
# How much USD borrowed relative to net USD deposited?
net_deposit_usd = wallet_features['net_deposit_amountUSD']
# Handle cases where net_deposit is zero or negative - assign a high ratio
# If net deposit <= 0 and they borrowed, it's high risk. Assign a high fixed value (e.g., 10).
# If net deposit > 0, calculate the ratio.
wallet_features['borrow_to_deposit_ratio_usd'] = np.where(
    (net_deposit_usd <= epsilon) & (total_borrowed_usd > epsilon),
    10.0, # Assign high ratio if borrowing with no/negative net deposit
    np.where(
        net_deposit_usd > epsilon,
        total_borrowed_usd / net_deposit_usd, # Calculate ratio normally
        0.0 # Assign 0 if not borrowing or if net deposit is positive but no borrows
    )
)
# Cap the ratio to prevent extreme values (e.g., cap at 10x)
wallet_features['borrow_to_deposit_ratio_usd'] = wallet_features['borrow_to_deposit_ratio_usd'].clip(0, 10)


# --- Add simple boolean flags ---
wallet_features['has_borrowed'] = (wallet_features.get(borrow_count_col, 0) > 0).astype(int) # 1 if true, 0 if false


# --- Clean up potential infinities or NaNs (though handled above) ---
wallet_features.replace([np.inf, -np.inf], np.nan, inplace=True)
# Re-check NaNs in calculated ratio columns and fill with appropriate default (e.g., 0)
ratio_cols = ['repayment_ratio_usd', 'borrow_to_deposit_ratio_usd']
for col in ratio_cols:
    if col in wallet_features.columns:
        wallet_features[col] = wallet_features[col].fillna(0)


print("Feature Engineering Complete.")
print(f"Wallet Features DataFrame shape: {wallet_features.shape}")

# --- Display Sample Features ---
print("\n--- Sample Wallet Features (First 5 Rows): ---")
print(wallet_features.head())

print("\n--- Wallet Features Info (wallet_features.info()): ---")
wallet_features.info(verbose=True, show_counts=True)

# --- Display Summary Statistics for Key Features ---
print("\n--- Summary Statistics for Key Features: ---")
key_feature_cols = [
    'wallet_lifespan_days', 'total_transactions', 'transactions_per_day',
    'unique_assets_interacted', 'net_deposit_amountUSD',
    'repayment_ratio_usd', 'borrow_to_deposit_ratio_usd', 'has_borrowed'
]
# Filter to only include columns that actually exist in the dataframe
key_feature_cols = [col for col in key_feature_cols if col in wallet_features.columns]
print(wallet_features[key_feature_cols].describe())

Starting Feature Engineering...
Input DataFrame 'df' shape: (120000, 7)
Grouping data by wallet address...
Initial aggregation complete.
Calculating time-based features...
Time-based features calculated.
Calculating counts and sums per transaction type...
Merging aggregated features...
Aggregated features merged.
Calculating derived behavioral features...
Feature Engineering Complete.
Wallet Features DataFrame shape: (13484, 19)

--- Sample Wallet Features (First 5 Rows): ---
                                                    first_seen  \
wallet_address                                                   
0x00000000af5a61acaf76190794e3fdf1289288a1 2019-08-14 17:21:40   
0x00000000b1786c9698c160d78232c78d6f6474fe 2020-02-12 20:48:55   
0x000000aaee6a496aaf7b7452518781786313400f 2019-07-02 10:53:55   
0x00000a6bbb35ea3f6f8dbc94801738f490c30a8d 2019-06-03 23:56:55   
0x000206732258d7511fa624127228e6a032718e62 2019-10-01 08:28:37   

                                                     las

In [ ]:
# Cell 5: Scoring Model Implementation (Weighted Sum Approach)

from sklearn.preprocessing import MinMaxScaler

print("Implementing Scoring Model...")

# --- 1. Select Features for Scoring ---
# Choose features that represent the behavior principles.
# Based on .describe() and our goals (rewarding stability, penalizing risk).
# Note: No direct 'liquidation' features was found.
feature_columns_for_scoring = [
    # Positive indicators (Good Behavior)
    'wallet_lifespan_days',       # Longer participation = more stable/committed (usually)
    'net_deposit_amountUSD',    # Higher net deposit = more collateral / value provided
    'repayment_ratio_usd',        # Higher ratio = responsible borrowing
    'unique_assets_interacted',   # More assets might mean deeper engagement (use lower weight)

    # Negative indicators (Risky/Bad Behavior)
    'borrow_to_deposit_ratio_usd',# Higher ratio = higher leverage/risk
    'transactions_per_day'        # Very high might indicate bot activity (penalize slightly)

    # Neutral/Contextual (Use with care or lower weight)
    # 'total_transactions' - volume isn't inherently good/bad, covered by freq.
]

# Verify selected columns exist in the DataFrame
existing_features = [col for col in feature_columns_for_scoring if col in wallet_features.columns]
print(f"Using these features for scoring: {existing_features}")

# Create a DataFrame with only the features needed for scoring
scoring_data = wallet_features[existing_features].copy()


# --- 2. Normalize Features ---
# Scale features to a 0-1 range. This makes weighting straightforward.
scaler = MinMaxScaler()
scoring_data_normalized = scaler.fit_transform(scoring_data)

# Convert back to DataFrame with original index and column names
scoring_data_normalized = pd.DataFrame(scoring_data_normalized,
                                       columns=existing_features,
                                       index=scoring_data.index)

print("\nNormalized features (sample):")
print(scoring_data_normalized.head())


# --- 3. Define Weights ---
# Weights reflect importance and direction (+ good, - bad).
# Consider the distribution from .describe() when setting weights.
weights = {
    # Positive weights:
    'wallet_lifespan_days': 0.20,  # Reward longer participation noticeably.
    'net_deposit_amountUSD': 0.25, # Reward higher net value provided (strong indicator). Needs scaling.
    'repayment_ratio_usd': 0.30,   # High importance: Paying back loans is key credit behavior.
    'unique_assets_interacted': 0.05,# Lower importance, but rewards broader engagement slightly.

    # Negative weights:
    'borrow_to_deposit_ratio_usd': -0.15, # Penalize high leverage/risk.
    'transactions_per_day': -0.05     # Small penalty for potentially bot-like high frequency.
}
# Note: Weights don't need to sum to 1. The final scaling have to handle the range.

print("\nDefined Weights:")
for feature, weight in weights.items():
    if feature in existing_features:
        print(f"- {feature}: {weight}")


# --- 4. Calculate Raw Score ---
# Apply weights to the NORMALIZED features
raw_score = 0
print("\nApplying weights to normalized features...")
for feature, weight in weights.items():
    if feature in scoring_data_normalized.columns:
        raw_score += scoring_data_normalized[feature] * weight
    else:
        print(f"Warning: Feature '{feature}' in weights not found in scoring data.")

wallet_features['raw_score'] = raw_score # Add raw score to the main features DataFrame
print("Raw score calculated.")


# --- 5. Scale Score to 0-100 ---
# Use MinMaxScaler again on the raw scores for final presentation range.
score_scaler = MinMaxScaler(feature_range=(0, 100))

# Reshape raw_score for the scaler (it expects 2D array)
wallet_features['final_score'] = score_scaler.fit_transform(wallet_features[['raw_score']])

#  Round the final score for cleaner presentation
wallet_features['final_score'] = wallet_features['final_score'].round(2)

print("Final score calculated and scaled to 0-100.")


# --- Display Score Results ---
print("\n--- Sample Scores (First 5 Rows): ---")
# Show relevant features alongside scores for context
display_cols = existing_features + ['raw_score', 'final_score']
print(wallet_features[display_cols].head())

print("\n--- Final Score Distribution Summary: ---")
print(wallet_features['final_score'].describe())

Implementing Scoring Model...
Using these features for scoring: ['wallet_lifespan_days', 'net_deposit_amountUSD', 'repayment_ratio_usd', 'unique_assets_interacted', 'borrow_to_deposit_ratio_usd', 'transactions_per_day']

Normalized features (sample):
                                            wallet_lifespan_days  \
wallet_address                                                     
0x00000000af5a61acaf76190794e3fdf1289288a1              0.395210   
0x00000000b1786c9698c160d78232c78d6f6474fe              0.095808   
0x000000aaee6a496aaf7b7452518781786313400f              0.781437   
0x00000a6bbb35ea3f6f8dbc94801738f490c30a8d              0.104790   
0x000206732258d7511fa624127228e6a032718e62              0.000000   

                                            net_deposit_amountUSD  \
wallet_address                                                      
0x00000000af5a61acaf76190794e3fdf1289288a1               0.270403   
0x00000000b1786c9698c160d78232c78d6f6474fe               0.271447

In [ ]:
# Cell 6: Generate Output Files

print("Generating Output Files...")

# --- 1. Prepare Final Output Data ---
# Sort the wallet_features DataFrame by final_score in descending order
final_scores_sorted = wallet_features.sort_values(by='final_score', ascending=False)

print(f"Total wallets scored: {len(final_scores_sorted)}")

# --- 2. Create Top 1,000 Scores CSV ---
# Select the top 1,000 rows
top_1000_scores_df = final_scores_sorted.head(1000)

# Create the specific output format: wallet_address, final_score
# Reset index to bring 'wallet_address' back as a column
output_csv_data = top_1000_scores_df.reset_index()

# Select and rename columns for the final CSV
output_csv_data = output_csv_data[['wallet_address', 'final_score']]

# Define the output filename
OUTPUT_CSV_FILENAME = 'top_1000_scores.csv'

# Save the CSV file
try:
    output_csv_data.to_csv(OUTPUT_CSV_FILENAME, index=False)
    print(f"Successfully saved top 1,000 wallet scores to '{OUTPUT_CSV_FILENAME}'")
except Exception as e:
    print(f"---!!! ERROR saving CSV file '{OUTPUT_CSV_FILENAME}': {e} !!!---")

# Display the top 10 wallets from the CSV data
print("\n--- Top 10 Wallets (from CSV data): ---")
print(output_csv_data.head(10))


# --- 3. Save ALL Wallet Scores and Features (for Analysis) ---
# This file will contain all wallets, all features, and the final score.
# It's useful for picking the bottom 5 and analyzing both top/bottom wallets.
ALL_SCORES_FEATURES_FILENAME = 'all_wallet_scores_and_features.csv'

try:
    # Reset index to make wallet_address a column before saving
    all_scores_to_save = final_scores_sorted.reset_index()
    all_scores_to_save.to_csv(ALL_SCORES_FEATURES_FILENAME, index=False)
    print(f"\nSuccessfully saved all wallet scores and features to '{ALL_SCORES_FEATURES_FILENAME}'")
except Exception as e:
    print(f"---!!! ERROR saving CSV file '{ALL_SCORES_FEATURES_FILENAME}': {e} !!!---")


print("\nOutput generation complete.")

Generating Output Files...
Total wallets scored: 13484
Successfully saved top 1,000 wallet scores to 'top_1000_scores.csv'

--- Top 10 Wallets (from CSV data): ---
                               wallet_address  final_score
0  0xc1852f917835a9f2f97112672bc5c8afd1f21dc3       100.00
1  0xc95be28595eb64485dac63ad14f15e41c94ee468        87.86
2  0x003c52a71c887461087154eccced08cb1c5384a5        84.54
3  0x124e1fafcadc2c017a17b4bbbbfff3867b7dee35        83.72
4  0x52405eff99dd4c2015463dc511573344f2e2e8d7        83.58
5  0xd19604efb40ece1efd6df5de6d3892112b51e99a        83.30
6  0x268a1b7ecc1fe1fab1ee32a7e61e3b7810bad4a5        82.90
7  0xf859a1ad94bcf445a406b892ef0d3082f4174088        82.58
8  0x869a032b284481f4ad359ac75cd017eddee5c23c        81.80
9  0x40e652fe0ec7329dc80282a6db8f03253046efde        81.67

Successfully saved all wallet scores and features to 'all_wallet_scores_and_features.csv'

Output generation complete.


In [ ]:
# Cell 7: Prepare for Wallet Analysis (Data Extraction)

print("--- Preparing Data for Wallet Analysis Document ---")

# Ensure the data is sorted by score (it should be from Cell 6)
final_scores_sorted = wallet_features.sort_values(by='final_score', ascending=False)

# --- Get Top 5 Wallets ---
top_5_wallets = final_scores_sorted.head(5)
print("\n--- Top 5 Scoring Wallets ---")
print(f"Addresses: {top_5_wallets.index.tolist()}")

# --- Get Bottom 5 Wallets ---
# Ensure we have at least 10 wallets to select from
if len(final_scores_sorted) >= 10:
    bottom_5_wallets = final_scores_sorted.tail(5)
    print("\n--- Bottom 5 Scoring Wallets ---")
    print(f"Addresses: {bottom_5_wallets.index.tolist()}")
else:
    print("\nWarning: Fewer than 10 wallets scored, cannot select distinct bottom 5.")
    # Handle this case if necessary, e.g., select fewer or all remaining.
    bottom_5_wallets = final_scores_sorted.tail(max(0, len(final_scores_sorted)-5))


# --- Display Features for Analysis Wallets ---
# Combine top and bottom wallets for easy viewing
analysis_wallets_df = pd.concat([top_5_wallets, bottom_5_wallets])

# Select the most relevant features used for scoring + the final score
# Use the 'existing_features' list from Cell 5 for consistency
analysis_display_cols = existing_features + ['final_score']

print("\n--- Features for Top 5 and Bottom 5 Wallets: ---")

print(analysis_wallets_df[analysis_display_cols])

print("\n--- End of Notebook Execution ---")

--- Preparing Data for Wallet Analysis Document ---

--- Top 5 Scoring Wallets ---
Addresses: ['0xc1852f917835a9f2f97112672bc5c8afd1f21dc3', '0xc95be28595eb64485dac63ad14f15e41c94ee468', '0x003c52a71c887461087154eccced08cb1c5384a5', '0x124e1fafcadc2c017a17b4bbbbfff3867b7dee35', '0x52405eff99dd4c2015463dc511573344f2e2e8d7']

--- Bottom 5 Scoring Wallets ---
Addresses: ['0x4b60f8b109bbc4ae3110f575f71e3eefba5d19ee', '0x3ee129d7370b53c8f4c6253c8d8cd4f47c800a5d', '0x555187752ef6d73758862b5d364aab362c996d0e', '0x3e292308f6fc7fb9c352c94609913b629b0e7088', '0xcf9060b126ed8995403d876e0f0bc3a1237bd8f9']

--- Features for Top 5 and Bottom 5 Wallets: ---
                                            wallet_lifespan_days  \
wallet_address                                                     
0xc1852f917835a9f2f97112672bc5c8afd1f21dc3                   237   
0xc95be28595eb64485dac63ad14f15e41c94ee468                   311   
0x003c52a71c887461087154eccced08cb1c5384a5                   292   
0x124e1fa